In [2]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [3]:
# Đọc dữ liệu
df_train = pd.read_csv("../clean_data/clean_data_train/encoded_data.csv")
df_test = pd.read_csv("../clean_data/clean_data_test/encoded_data.csv")

In [6]:
# Tách nhãn và đặc trưng 
X = df_train.drop(columns=["Phân Loại"])  # đặc trưng
y = df_train["Phân Loại"]                 # nhãn phân loại

# Xác định các cột số và phân loại
numeric_cols = X.select_dtypes(include="number").columns.tolist()
categorical_cols = X.select_dtypes(include="object").columns.tolist()

In [7]:
numeric_pipeline = Pipeline([
    ('select', SelectKBest(score_func=f_classif, k=10))
])

preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, numeric_cols)
], remainder='passthrough')  # giữ lại các cột đã mã hóa

# Fit pipeline và biến đổi dữ liệu
X_transformed = preprocessor.fit_transform(X, y)

# Lấy điểm số f_classif của các đặc trưng số
select_kbest = preprocessor.named_transformers_['num'].named_steps['select']
scores = select_kbest.scores_

# Tạo bảng điểm đặc trưng
feature_scores = pd.DataFrame({
    'Feature': numeric_cols,
    'Score (f_classif)': scores
}).sort_values(by='Score (f_classif)', ascending=False)

print("Bảng xếp hạng đặc trưng số theo f_classif:")
print(feature_scores)

# Lấy tên top 10 đặc trưng số được chọn
selected_mask = select_kbest.get_support()
selected_features = [f for f, keep in zip(numeric_cols, selected_mask) if keep]

print("\nTop 10 đặc trưng số được chọn:")
print(selected_features)

Bảng xếp hạng đặc trưng số theo f_classif:
              Feature  Score (f_classif)
15           Hãng GPU         115.864682
1              Độ Dày         114.211237
13       Độ Phân Giải          88.065910
2         Trọng Lượng          61.663581
7             Giá Bán          44.542917
11              Cache          42.591524
9              Số Lõi          41.354839
5      Dung Lượng Pin          27.801474
0       Hãng Sản Xuất          27.104886
4              Bộ Nhớ          26.408958
6        Hệ Điều Hành          23.189814
14  Số Cổng Giao Tiếp          13.642868
8            Hãng CPU           8.661398
10   Xung nhịp tối đa           7.782629
12      Loại Màn Hình           7.620671
3                 RAM           7.569242

Top 10 đặc trưng số được chọn:
['Hãng Sản Xuất', 'Độ Dày', 'Trọng Lượng', 'Bộ Nhớ', 'Dung Lượng Pin', 'Giá Bán', 'Số Lõi', 'Cache', 'Độ Phân Giải', 'Hãng GPU']


In [32]:
df_train[selected_features + ["Phân Loại"]].to_csv("data_train.csv", index=False)
df_test[selected_features + ["Phân Loại"]].to_csv("data_test.csv", index=False)